In [1]:
import weave
from set_env import set_env
import nest_asyncio
from evalforge.combined_scorer import AssertionScorer
from evalforge.evalforge import convert_datapoint_to_example
from evalforge.combined_scorer import predict_passthrough
import asyncio


ImportError: cannot import name 'predict_passthrough' from 'evalforge.evalforge' (/Users/anishshah/Documents/Manual Library/GitHub(1)/improve-evals/evalforge/evalforge.py)

In [ ]:
set_env("OPENAI_API_KEY")
set_env("WANDB_API_KEY")
print("Env set")

In [ ]:
try:
    import IPython
    in_jupyter = True
except ImportError:
    in_jupyter = False
if in_jupyter:
    nest_asyncio.apply()

In [ ]:
import random
weave.init(f"forged_test_{random.randint(0, 10000000)}")

In [ ]:
TEST_TASK = "medical"
if TEST_TASK == "medical":
    all_data = weave.ref("weave:///a-sh0ts/medical_data_results/object/medical_data_annotations:7GcCtWgyPTWtKY48Z7v5VxwCNZXTTTpSMbmubAbyHT8").get()
    data = random.sample(all_data, 10)
elif TEST_TASK == "product":
    pass
else:
    all_data = data = [
        ({"text": "Summarize the impact of climate change on polar bears."}, {"text": "Climate change is reducing sea ice, which polar bears rely on for hunting seals."}, 1, "Accurate and relevant."),
        ({"text": "Explain the process of photosynthesis."}, {"text": "Photosynthesis is the process by which plants use sunlight to synthesize foods from carbon dioxide and water."}, 1, "Correct and detailed."),
        ({"text": "What are the main causes of the American Civil War?"}, {"text": "The main causes were slavery, states' rights, and economic differences."}, 1, "Concise and accurate."),
        ({"text": "Describe the symptoms of COVID-19."}, {"text": "COVID-19 is caused by a virus that originated in bats."}, 0, "Irrelevant and incorrect."),
        ({"text": "What is the significance of the Magna Carta?"}, {"text": "The Magna Carta was a document that limited the power of the king and established certain legal rights."}, 1, "Historically accurate and relevant.")
    ]

In [ ]:
# Ideal
# forged_judge = weave.ref("weave:///a-sh0ts/evalgen_test_363037/object/final_judge:M6GWs63FKPj8KNNOd56ASr2KwuwNn7YbAAFZdixSA0I").get()

In [ ]:
# Copium
forged_judge = AssertionScorer()
forged_judge.import_assertions("forged_judge")


In [ ]:
finalized_task_description = "The task requires transforming detailed medical dialogues between a doctor and a patient into structured medical notes in a standardized bullet-point list format. The dialogues provide comprehensive information on the patient's medical history, current symptoms, examination findings, familial history, allergies, medications, and future healthcare plans. The output should be a concise, structured list containing the mandatory fields: Chief complaint, History of present illness, Physical examination findings, Symptoms, New medications with dosages, and Follow-up instructions. Missing data should be marked as 'N/A'. The process demands the accurate extraction and summary of relevant medical information while strictly omitting any personally identifiable information (PII) to ensure privacy, substituting terms like 'the patient' for specific identifiers. Notes should maintain consistent punctuation and formatting, with new medications distinctly separated from existing ones. Evaluation focuses on accuracy, completeness, adherence to structure, exclusion of PII, consistent punctuation, and correct labeling of missing data. This task aims to produce anonymized medical notes that support healthcare delivery without compromising patient confidentiality, adhering to relevance, conciseness, and a 150-word limit for summaries if necessary."

In [ ]:
all_annotation_examples = convert_datapoint_to_example(finalized_task_description, all_data)
evaluation = weave.Evaluation(
    scorers=[forged_judge],
    dataset=all_annotation_examples,
)


final_judge_results = asyncio.run(evaluation.evaluate(predict_passthrough))